# Análise exploratória de dados com Pandas, Plotly e spaCy


- Objetivo: Obter uma visão geral do acervo do periódico **Africa: Journal of the International African Institute** e de sua contribuição para os estudos africanos no recorte temporal de 1920 a 1960. A partir dessa análise inicial, poderemos identificar os autores que mais publicaram, em quais idiomas e quais os espaços geográficos estavam sendo mencionados nos trabalhos. 

- A análise utiliza metadados coletados da base bibliográfica do [JSTOR](https://www.jstor.org), a partir da ferramenta [Constellate](https://constellate.org). 

## Módulos e Bibliotecas utilizadas:
- [Pandas](https://pandas.pydata.org/docs/)
- [Plotly Express](https://plotly.com/python/plotly-express/)
- [spaCy](https://spacy.io)

In [45]:
# Importa as bibliotecas e módulos a serem utilizadas na análise exploratória de dados

import pandas as pd 
import plotly.express as px
import spacy

## Importação do dataset e visão geral dos dados

In [46]:
# Importa o dataset criado com o Constellate.org para um Dataframe Python

df = pd.read_csv('csv/metadata.csv')

In [47]:
# Exibe as informações gerais do dataframe, como quantidade e nome das colunas, o número de
# valores não-nulos, e os tipos de dados de cada coluna. 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2555 entries, 0 to 2554
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2555 non-null   object 
 1   title               2555 non-null   object 
 2   isPartOf            2555 non-null   object 
 3   publicationYear     2555 non-null   int64  
 4   doi                 0 non-null      float64
 5   docType             2555 non-null   object 
 6   docSubType          2555 non-null   object 
 7   provider            2555 non-null   object 
 8   collection          0 non-null      float64
 9   datePublished       2555 non-null   object 
 10  issueNumber         2555 non-null   int64  
 11  volumeNumber        2555 non-null   int64  
 12  url                 2555 non-null   object 
 13  creator             1956 non-null   object 
 14  publisher           2555 non-null   object 
 15  language            2555 non-null   object 
 16  pageSt

In [48]:
# Exibe os primeiros cinco resultados do dataframe. 
# Esse passo é importante para termos uma visão geral dos dados que possuímos. 

df.head(5)

,id,title,isPartOf,publicationYear,doi,docType,docSubType,provider,collection,datePublished,...,url,creator,publisher,language,pageStart,pageEnd,placeOfPublication,wordCount,pageCount,outputFormat
0,http://www.jstor.org/stable/1156397,Review Article,Africa: Journal of the International African I...,1943,NaN,article,book-review,jstor,NaN,1943-04-01,...,http://www.jstor.org/stable/1156397,E. O. Ashton,Cambridge University Press,eng,102,102,NaN,690,1,unigram; bigram; trigram
1,http://www.jstor.org/stable/3180594,INZUIKIƷI,Africa: Journal of the International African I...,1935,NaN,article,research-article,jstor,NaN,1935-10-01,...,http://www.jstor.org/stable/3180594,Edwin W. Smith,Cambridge University Press,eng,473,480,NaN,3505,8,unigram; bigram; trigram
2,http://www.jstor.org/stable/1155333,Review Article,Africa: Journal of the International African I...,1932,NaN,article,book-review,jstor,NaN,1932-01-01,...,http://www.jstor.org/stable/1155333,A. Werner,Cambridge University Press,eng,95,97,NaN,1429,3,unigram; bigram; trigram
3,http://www.jstor.org/stable/1155810,Review Article,Africa: Journal of the International African I...,1930,NaN,article,book-review,jstor,NaN,1930-04-01,...,http://www.jstor.org/stable/1155810,Henri Labouret,Cambridge University Press,fre,247,248,NaN,909,2,unigram; bigram; trigram
4,http://www.jstor.org/stable/1156777,Review Article,Africa: Journal of the International African I...,1956,NaN,article,book-review,jstor,NaN,1956-01-01,...,http://www.jstor.org/stable/1156777,Jan Vansina,Cambridge University Press,fre,91,92,NaN,1450,2,unigram; bigram; trigram


In [49]:
# Exibe uma lista das colunas do dataframe

df.columns

Index(['id', 'title', 'isPartOf', 'publicationYear', 'doi', 'docType',
       'docSubType', 'provider', 'collection', 'datePublished', 'issueNumber',
       'volumeNumber', 'url', 'creator', 'publisher', 'language', 'pageStart',
       'pageEnd', 'placeOfPublication', 'wordCount', 'pageCount',
       'outputFormat'],
      dtype='object')


## Limpeza de dados

- Nesta etapa, iremos tratar os dados. Renomearemos e excluiremos os elementos necessários, para que o dataset apenas inclua os dados que pretenderemos trabalhar. 


In [50]:
# Exclui as colunas não necessárias e/ou duplicadas

df = df.drop(['isPartOf', 'doi','provider', 'collection', 'pageStart', 'pageEnd', \
'placeOfPublication', 'url', 'outputFormat'], axis=1)

In [51]:
# Renomeia nomes de colunas

df.rename(columns={'publicationYear': 'year','datePublished':'date', 'creator': 'author'},\
    inplace=True)



- Iremos selecionar as linhas de dados que serão úteis à nossa análise. Nesta etapa, apenas iremos trabalhar com as categorias de artigos originais ('research-article') e resenhas de livros(book-review). Artigos de miscelânea e notícias não serão úteis à nossa análise atual. 



In [52]:
# Conta os valores dos subtipos de textos publicados 

df['docSubType'].value_counts()

book-review         1200
research-article     719
misc                 595
news                  41
Name: docSubType, dtype: int64

In [53]:
# Cria um dataset apenas de artigos originais

articles = df[df['docSubType'] == 'research-article']

In [54]:
# Exibe as informações gerais do dataset criado

articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 719 entries, 1 to 2554
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            719 non-null    object
 1   title         719 non-null    object
 2   year          719 non-null    int64 
 3   docType       719 non-null    object
 4   docSubType    719 non-null    object
 5   date          719 non-null    object
 6   issueNumber   719 non-null    int64 
 7   volumeNumber  719 non-null    int64 
 8   author        707 non-null    object
 9   publisher     719 non-null    object
 10  language      719 non-null    object
 11  wordCount     719 non-null    int64 
 12  pageCount     719 non-null    int64 
dtypes: int64(5), object(8)
memory usage: 78.6+ KB


In [55]:
# Cria um dataset com a contagem de artigos por autor

authors = articles['author'].value_counts()\
    .reset_index(name='total')\
    .rename(columns={'index':'author'})\
    .sort_values(by='total', ascending=False)

In [56]:
# Exibe o dataset criado

authors

,author,total
0,E. E. Evans-Pritchard,19
1,Henri Labouret,8
2,M. D. W. Jeffreys,7
3,S. F. Nadel,7
4,L. P. Mair,6
...,...,...
242,Jean Comhaire,1
241,D. Zahan; Solange De Ganay,1
240,Thérèse Rivière,1
239,F. J. Pedler,1


In [57]:
# Cria um gráfico com o número de publicações por autor

fig = px.bar(authors.head(35), x='author', y='total')

In [58]:
# Configura o layout do gráfico

fig.update_layout(
    title='<b>Número de publicações originais por autor</b>',
    xaxis_title='Autores',
    yaxis_title='Total de artigos',
    template = 'plotly_dark'
)

# Configura o traçado

fig.update_traces(
    hovertemplate='<b>%{x}</b><br>Número de publicações originais por autor: %{y}'
)
# Exibe o Gráfico

fig.show()


In [59]:
# Cria um dataframe com a contagem dos artigos por idioma de publicação

idiomas_df = articles['language'].value_counts()\
    .reset_index(name='total')\
    .rename(columns={'index':'idioma'})\
    .sort_values(by='total', ascending=False)

In [60]:
# Substitui as abreviações pelos nomes dos idiomas em português

idiomas_df['idioma'] = idiomas_df['idioma'].replace({'eng': 'Inglês', 'ger': 'Alemão', 'fre': 'Francês',\
    'swa': 'Suaíli', 'ibo': 'Igbo'})

In [61]:
# Exibe o dataframe

idiomas_df

,idioma,total
0,Inglês,568
1,Francês,114
2,Alemão,34
3,Suaíli,2
4,Igbo,1


In [62]:
# Cria um gráfico com a contagem dos artigos por idioma de publicação

fig2 = px.line(idiomas_df, x= 'idioma', y='total')


In [63]:
# Configura o layout do gráfico

fig2.update_layout(
    title='<b>Número de publicações originais por idioma</b>',
    xaxis_title='Idioma',
    yaxis_title='Total de artigos',
    template = 'plotly_dark'
)

# Configura o traçado do gráfico
fig2.update_traces(
    hovertemplate='<b>%{x}</b><br>Número de publicações: %{y}',
    mode='markers+lines'
)
# Exibe o gráfico

fig2.show()

In [64]:
# Cria um dataframe com os metadados das resenhas de livros

bookreview= df[df['docSubType'] == 'book-review']

In [65]:
# Exibe as informações gerais do dataframe

bookreview.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 2553
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1200 non-null   object
 1   title         1200 non-null   object
 2   year          1200 non-null   int64 
 3   docType       1200 non-null   object
 4   docSubType    1200 non-null   object
 5   date          1200 non-null   object
 6   issueNumber   1200 non-null   int64 
 7   volumeNumber  1200 non-null   int64 
 8   author        1193 non-null   object
 9   publisher     1200 non-null   object
 10  language      1200 non-null   object
 11  wordCount     1200 non-null   int64 
 12  pageCount     1200 non-null   int64 
dtypes: int64(5), object(8)
memory usage: 131.2+ KB


In [66]:
# Cria um dataframe com a frequência de publicações de resenhas por autor

authors_br = bookreview['author'].value_counts()\
    .reset_index(name='total')\
    .rename(columns={'index':'author'})\
    .sort_values(by='total', ascending=False)

In [67]:
#Exibe o dataframe

authors_br

,author,total
0,D. Westermann,58
1,Henri Labouret,52
2,Edwin W. Smith,40
3,L. P. Mair,34
4,A. N. Tucker,22
...,...,...
282,J. F. Carrington,1
281,F. Lukyn Williams,1
280,H. G. Anderson,1
279,G. Tillion,1


In [82]:
# Gera um gráfico a partir do dataframe criado 

fig3 = px.bar(authors_br.head(30), x= 'author', y='total')


In [83]:
# Configura o layout do gráfico

fig3.update_layout(
    title='<b>Número de resenhas de livros por autor</b>',
    xaxis_title='Autores',
    yaxis_title='Total de resenhas',
    template = 'plotly_dark'
)

# Configura o traçado do gráfico
fig3.update_traces(
    hovertemplate='<b>%{x}</b><br>Número de resenhas publicadas: %{y}'
)

# Exibe o gráfico criado
fig3.show()


## Análise exploratória dos títulos das publicações com spaCy

- Nesta etapa, iremos utilizar na análise o modelo disponível na biblioteca do spaCy ['en_core_web_sm'](https://github.com/explosion/spacy-models/releases/tag/en_core_web_sm-3.2.0), treinado com textos escritos na língua inglesa de blogs, notícias e comentários provenientes da web. 


In [84]:
# Salva os títulos dos artigos em um arquivo .txt 

titulos = articles['title']

titulos.to_csv(r'csv/titulos.txt', header=None, index=None, sep=',', mode='a')

In [85]:
# Carrega o modelo do spacy 'en_core_web_sm' 

nlp = spacy.load('en_core_web_sm')

In [86]:
# Lê o texto

text = open('csv/titulos.txt', 'r').read()

# Processa o texto em uma entidade 'doc'
doc = nlp(text)

In [87]:
# Cria uma lista apenas com as entidades do tipo GPE e LOC (Entidades Geopolíticas e Localização) através 
# de controladores de fluxo "if" e "for"

locations = []

for named_entity in doc.ents:
    if named_entity.label_ == 'GPE' and 'LOC':
        itens = named_entity
        locations.append(itens)



In [88]:
# Exibe a lista

locations

[Poland,
 Central Africa,
 West Africa,
 Anthropological Field-Worker,
 West Africa,
 East Africa,
 Yoruba,
 Southern Nigeria,
 Uganda,
 MONGO,
 Colonies,
 Gambia,
 Senegal,
 Ethiopia,
 West Africa,
 Bemba,
 Eastern Africa,
 Ahnenkult,
 Kamerun,
 East Africa,
 Nigeria,
 Tikar Chiefdom,
 Dogon,
 Clan,
 Chieftainship,
 Yoruba,
 Togoland,
 Yoruba,
 Bemba,
 Somaliland,
 Häuptlingswürde,
 Proverb,
 Kenya,
 Nigeria,
 Katanga,
 Northern Nigeria,
 Congo,
 India,
 Northern Nigeria,
 Ghana,
 Morocco,
 Swaziland,
 Abyssinia,
 Dogon,
 Oberguinea,
 South Africa,
 Sudan,
 East Africa,
 West Africa,
 Dahomey,
 Modern Africa,
 West Africa,
 Central Africa,
 Nigeria,
 South Africa,
 Ethiopia,
 Umor,
 West Africa,
 the Northern Province,
 Nyasaland,
 Benin City,
 East Africa,
 Central Africa,
 West Africa,
 Devinettes,
 Sudan Village Following Locust Visitation,
 the Netherlands East Indies,
 Northern Nigeria,
 Lugbara,
 Vengeance,
 West Africa,
 East Africa,
 South Africa,
 East Africa,
 Nineteenth Cen

- Apesar da alta precisão do modelo ([0.85](https://spacy.io/models/en) estimado de precisão de entidades nomeadas), alguns resultados não correspondem à pesquisa de termos relacionados à entidades geográficas, como os resultados 'Anthropological Field-Worker' e 'Colonies', 'Nineteenth Centuries' ou 'Yoruba' (que se refere à uma língua e a um grupo etnolinguístico e não a um local).  
- Utilizamos na pesquisa as categorias 'GPE'(Entidades geopolíticas) e 'LOC' (localizações) para filtrar os resultados que mencionam entidades geográficas.

In [91]:
# Cria um dataframe a partir do arquivo .csv criado com a lista de localizações

loc = pd.read_csv('csv/loc.csv')

In [92]:
# Exibe o dataframe criado com a lista de localizações 
loc

,Localizações,Unnamed: 1
0,Poland,NaN
1,Central Africa,NaN
2,West Africa,NaN
3,Anthropological Field-Worker,NaN
4,West Africa,NaN
...,...,...
123,Melanesia,NaN
124,Palestine,NaN
125,Yoruba,NaN
126,Dahomey,NaN


In [93]:
# Exclui a coluna desnecessária do dataframe

loc = loc.drop(columns='Unnamed: 1')

In [94]:
# Cria um dataframe com a frequência das entidades geográficas identificadas

df_loc_freq = loc['Localizações'].value_counts()\
    .reset_index(name='total')\
    .rename(columns={'index':'Localizações'})\
    .sort_values(by='total', ascending=False)
    
# Exibe o dataframe

df_loc_freq

,Localizações,total
0,West Africa,10
1,East Africa,10
2,Nigeria,7
3,Yoruba,5
4,South Africa,5
...,...,...
41,Johannesburg,1
42,Melanesia,1
43,Palestine,1
44,Lugbara,1


In [95]:
# Cria um gráfico com a frequência das entidades geográficas mencionadas nos títulos dos artigos

fig4 = px.bar(df_loc_freq.head(20), x= 'Localizações', y='total')


In [96]:
# Configura o layout do gráfico

fig4.update_layout(
    title='<b>Número de menções a entidades geográficas nos títulos dos artigos</b>',
    xaxis_title='Entidades Geográficas',
    yaxis_title='Total de menções',
    template = 'plotly_dark'
)

# Configura o traçado

fig4.update_traces(
    hovertemplate='<b>%{x}</b><br>Número de menções: %{y}'
)

#Exibe o gráfico 

fig4.show()

## Exportação da Visualização dos Dados

In [97]:
# Salva os gráficos em .html

fig.write_html('graficos/fig.html')
fig2.write_html('graficos/fig2.html')
fig3.write_html('graficos/fig3.html')
fig4.write_html('graficos/fig4.html')
